# Chapter 5 Working with text¶
Julia as a Second Language, Erik Engheim

Topics

1. Representing text with the String type
2. Formatting text with lpad and rpad
3. Reading text from a keyboard or file
4. Writing text to the screen or a file
5. Creating a simple interactive program
*-----------------------------------------------------------------------------*

## Different ways of displaying text on the screen

Functions used:

- `print`
- `println`
- `printstyled`
- `rpad`
- `lpad`

### Print, println, and printstyled demos

In [ ]:
println("hello"); println("world")

In [ ]:
print("hello"); print(" world\n")

In [ ]:
println("hello\tworld")   # \t hprizontal tab

In [ ]:
println("hello \v world") # \v vertical tab

In [ ]:
println("hello \r world") # \r carriage return

In [ ]:
printstyled("hello world", color = :green) # keyword argument, :symbol

In [ ]:
printstyled(underline= true, "hello world", color = :yellow)

In [ ]:
?printstyled

All the colors are given as symbols. A symbol is very similar to a text string. It is often
used for text strings, which only matter to programmers and not users of your program.

In [ ]:
sym = Symbol("Hi")

In [ ]:
sym

### Printing multiple elements

In [ ]:
print("abc", 42, true, "xyz")

In [ ]:
string("abc", 42, true, "xyz")

## Printing multiple pizzas example

In [ ]:
pizzas = [                     # A Vector of tuples
("hawaiian", 'S', 10.5),
("mexicana", 'S', 13.0),
("hawaiian", 'L', 16.5),
("bbq chicken", 'L', 20.75),
("sicilian", 'S', 12.25),
("bbq chicken", 'M', 16.75),
("mexicana", 'M', 16.0),
("thai chicken", 'L', 20.75),
];

name(pz) = pz[1];
portion(pz) = pz[2];
price(pz) = pz[3];

In [ ]:
for pz in pizzas
    println(name(pz), " ", portion(pz), " ", price(pz))
end

#### Align with `lpad` and `rpad`

With the Julia padding functions you can specify that a text string should always be of
a given length. If the text you supply is smaller, it will get padded with a chosen character.
If no character is specified, the padding character defaults to space.

In [ ]:
lpad("ABC", 6, '-'), lpad("ABC", 6), lpad("1234567890", 6)

In [ ]:
rpad("ABC", 6, '-'), rpad("ABC", 6), rpad("1234567890", 6)

With lpad and rpad you can define how wide each column in your table should be
and add padding, such as spaces, wherever the supplied text string is shorter. In this
example, you’ll keep it simple and check what width **the widest strings in each column** would be:

In [ ]:
length("thai chicken") # width of pizza name column

In [ ]:
length("size") # width of pizza portion column

In [ ]:
max(length("16.75"), length("price")) # width of pizza price column

In [ ]:
"""
    simple alligned pizza table

"""

function simple_pizzatable(pizzas)
    pname = rpad("name", 12)
    psize = rpad("size", 4)
    pprice = rpad("price", 5)
    printstyled(pname, " ",
                psize, " ",
                pprice,
                color=:cyan)
    println()
    for pz in pizzas
        pname = rpad(name(pz), 12)
        psize = rpad(portion(pz), 4)
        pprice = lpad(price(pz), 5)
        println(pname, " ", psize, " ", pprice)
    end
end

In [ ]:
simple_pizzatable(pizzas)

Notice how the decimal points are not aligned. There are many ways to solve that, but in thenext example with trigonometric tables, you will make sure each number has the same number of decimals after the point.

#### Adding lines in print

Adding separating lines is actually quite simple. You simply use the symbol for a long
vertical line: '│': Unicode \U2502

In [ ]:
'\U2502' # define a Unicode character in Char using \U<hex><hex><hex><hex

In [ ]:
collect("─├┼┤") 

In [ ]:
collect("\U2500\U251C\U253C\U2524") # define Unicode chararters in String

In [ ]:
join(collect("\U2500\U251C\U253C\U2524"))

## Printing a trigonometric functions table

In [ ]:
"""

     Format number output with a desired number of digits after point

"""

n = length("-0.999")   # desired digit format: -d.ddd or d.ddd

function format(x)
    x = round(x, digits=3)
    if x < 0        
        rpad(x, n, '0')
    else
        rpad(x, n-1, '0')
    end
end

In [ ]:
format(-3.66689), format(-45.367)

In [ ]:
format(3.66689), format(45.367)

In [ ]:
println(format(3.66689))
println(format(45.367))

In [ ]:
"""

     Format number output with a desired number of digits after point

"""

n = length("-0.999")   # desired digit format: -d.ddd or d.ddd

function format(x)
    x = round(x, digits=3)
    if x < 0        
        rpad(x, n, '0')
    else
        rpad(x, n-1, '0')
    end
end

In [ ]:
"""

   Print sin and cos table in an range of degree values from 0
   (i.e. 0:inc:maxangle)

"""

function print_trigtable(inc, maxangle)
    print("│ ")
    printstyled("θ  ", color=:cyan)
    print(" │ ")
    printstyled(rpad("cos", n),
                color=:cyan)
    print(" │ ")
    printstyled(rpad("sin", n),
                color=:cyan)
    println(" │")
    angle = 0
    while angle <= maxangle
        rad = deg2rad(angle)
        cosx = format(cos(rad))
        sinx = format(sin(rad))
        print("│ ")
        print(lpad(angle, 3), " │ ",
        lpad(cosx, n), " │ ",
        lpad(sinx, n))
        println(" │")
        angle += inc
    end
end

In [ ]:
print_trigtable(15, 90)

## Reading and writing pizza sales to CSV files

A very common file format
for exchanging data between various types of spreadsheet applications and scientific applications is called `CSV`, which is short for Comma Separated Values. You will implement a store_pizzatable function to write pizza data in `CSV` format to a file and a load_pizzatable function to read the same `CSV` file. The following is an example of the pizza data CSV file format both functions will work with:

1. name,size,price
2. hawaiian,S,10.5
3. mexicana,S,13.0
4. hawaiian,L,16.5
5. bbq chicken,L,20.75
6. sicilian,S,12.25
7. bbq chicken,M,16.75
8. mexicana,M,16.0
9. thai chicken,L,20.75

The first line is referred to as the header. It gives a name to each column in the file. For each row you separate each value with a comma.

*-----------------------------------------------------------------------------------------------------------------------------*

Julia already has built-in functions for
this and very good external libraries, such as `CSV.jl` at `csv.juliadata.org`. However, your focus will be on learning the basics of reading and writing to text files; thus you will not be using external packages or functions.

## Writing pizza sales to a text file

You will define a simple function, store_pizzatable, which outputs pizza sales data as comma separated values:

In [ ]:
function store_pizzatable(io, pizzas)
    println(io, "name,size,price")
    for pz in pizzas
        println(io, name(pz), ",",
        portion(pz), ",",
        price(pz))
    end
end

This function should look familiar to you. What is new is that the println function is taking a new first argument named io. This presents some common pitfalls, so let me use this function incorrectly at first:

In [ ]:
store_pizzatable("-->", pizzas[1:3])

`println("hello")` is actually short for `println(stdout, "hello")`

`stdout` represents the default destination for anything printed..The default is your terminal window; however, the destination could be a file or even a network connection. You can try using stdout instead of the string "--->", although the result will be rather boring:

In [ ]:
store_pizzatable(stdout, pizzas[1:3])

It gets more interesting when you provide a file as a destination. To do that you need to create an IO object representing a file.

In [ ]:
io = open("pizza-sales.csv", "w")

In [ ]:
store_pizzatable(io, pizzas)

In [ ]:
# The connection to the file has to be closed when
# you are done. Reading and writing to a file can
# be buffered. Hence, unless you close, not all data
# will have necessarily been written yet.
close(io)

Where is the file `pizza-sales.csv` stored?

Answer: in the current working directory

Julia Base module provides basic interface to local filesystem:

**Reference** `Julia 1.9.0 Manual Chapter 50 Filesystem`

Some of useful filesystem functions include:

- `pwd()->AbstractString`

   Get the current working directory.
   
   
- `cd(dir::AbstractString=homedir())`

   Set the current working directory.
   
   
- `readdir(dir::AbstractString=pwd();
    join::Bool = false,
    sort::Bool = true,
) ->Vector{String}`

   Return the names in the directory dir or the current working directory if not given.
   

In [ ]:
pwd()

In [ ]:
readdir()

**Alternatively, you can use the shell dir command**

In windows, 

`shell> cmd /c dir`

lie this:

In [ ]:
;cmd /c dir

Moreove, you can the Windows shell cmd `type` (similar to `cat` in Linux)
to print the contents of a file to the cnsole like this:

`shell> cmd type <filename>`

In [ ]:
;cmd /c type pizza-sales.csv

## Reading pizza sales from a file


In [ ]:
io = open("pizza-sales.csv")

In [ ]:
line = readline(io)

In [ ]:
line = readline(io)

In [ ]:
close(io)

### Loading pizza sales data

In [ ]:
function load_pizzatable(io)
    pizzas = []
    readline(io)
    while !eof(io)
        pz = split(readline(io), ',')
        price = parse(Float64, pz[3])
        push!(pizzas, (pz[1], pz[1][1], price)) # (String,Char,Float64)
    end
    pizzas
end

In [ ]:
io = open("pizza-sales.csv")
pizzas_in = load_pizzatable(io)
close(io)

In [ ]:
pizzas_in

Another method of function load_pizzatable defined like this:

In [ ]:
function load_pizzatable(file::String)
    try
        io = open(file)
        
        pizzas = []
        readline(io)
        while !eof(io)
            pz = split(readline(io), ',')
            price = parse(Float64, pz[3])
            push!(pizzas, (pz[1], pz[1][1], price)) # (String,Char,Float64)
        end
        close(io)
        
        println("$file is read successfully.")
        pizzas
       
    catch
         println("$file does not exist!")
    end
end

In [ ]:
pizzas_in_2 = load_pizzatable("pizza-sales.csv");

In [ ]:
pizzas_in_2

In [ ]:
load_pizzatable("pizza-sales2.csv");